In [0]:
import os
import uuid
from databricks.sdk import WorkspaceClient

from helper import *

def copy_local_directory_to_external_location(local_directory, target_volume_location):
    """
    Copies all files and subdirectories from a local directory to Unity Catalog volume location using the Databricks SDK for Python.
    This function expects that your target location is added to volume and external location, using appropriate
    storage credential that has write permission to the provided location.

    Args:
        local_directory (str): The absolute path to the local directory you want to copy.
        target_volume_location (str): The absolute path of Unity Catalog volume location.

    Returns:
        bool: True if the copy was successful, False otherwise.
    """
    try:
        w = WorkspaceClient()

        logger.info(f"Preparing to copy files to the Unity Catalog volume path..")
        for root, _, files in os.walk(local_directory):
            for file in files:
                local_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(local_file_path, local_directory)
                dbfs_file_path = os.path.join(target_volume_location, relative_path).replace("\\", "/")
                with open(local_file_path, "rb") as f:
                    w.dbfs.upload(dbfs_file_path, f, overwrite=True)

        logger.info(f"Sucessfully copied files to target location!")
        return True
    except Exception as e:
        logging.error(f"Error copying '{local_directory}' to DBFS: {e}")
        return False

In [0]:
dbutils.widgets.text("target_volume_location", defaultValue="")
target_volume_location = dbutils.widgets.get("target_volume_location")
if target_volume_location == "":
    raise ValueError("Please provide a valid Unity Catalog Volume location to the 'target_volume_location' parameter. Format: /Volumes/mycatalog/myschema/myexternalvolume")
logger.info(f"Target Unity Catalog volume location path: '{target_volume_location}'")

# Upload JSON files on local storage to Unity Catalog volume
current_dir = os.getcwd()
data_path = os.path.join(current_dir, "data")
copy_local_directory_to_external_location(data_path, target_volume_location)

logger.info(f"Data copy process finished!")